In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
import torchaudio as ta
import torchaudio.functional as taf
import torchaudio.transforms as tat

print(torch.__version__)        
print(ta.__version__)

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio, display

import pandas as pd
import os
import pprint

from typing import *
import itertools
from collections import Counter

import numpy as np
from datetime import datetime
from lark.config import Config
from lark.learner import Learner
from lark.ops import Sig2Spec

1.8.1
0.8.0a0+e4e171a


In [3]:
import torch
# get list of models
torch.hub.list('zhanghang1989/ResNeSt', force_reload=True)
# load pretrained models, using ResNeSt-50 as an example
rn_model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)

Downloading: "https://github.com/zhanghang1989/ResNeSt/archive/master.zip" to /home/koen/.cache/torch/hub/master.zip
Using cache found in /home/koen/.cache/torch/hub/zhanghang1989_ResNeSt_master


In [21]:
cfg = Config(site='SSW', use_neptune=False, n_epochs=10, bs=64, lr=1e-2)

In [22]:
lrn = Learner("resnest", cfg)

In [23]:
model = torch.nn.Sequential(Sig2Spec(cfg, forward_as_image=True), rn_model)
model[1].fc = torch.nn.Linear(in_features=2048, out_features=len(cfg.labels), bias=True)

In [24]:
lrn.model = model.cuda()

In [25]:
lrn.loss_fn = torch.nn.BCEWithLogitsLoss().cuda()
# lrn.loss_fn = torch.nn.BCELoss().cuda()

In [26]:
# z = torch.randn([32, 1, 64, 513]).transpose(0, 1)
# y = torch.cat([z]*3).transpose(0, 1)
# y.shape


In [27]:
# batch = next(iter(lrn.tdl))

In [28]:
# batch[0].shape

In [29]:
# y_hat = lrn.model(batch[0].cuda())
# y = batch[1].cuda()

In [30]:
# lrn.loss_fn(y_hat, y)

In [31]:
lrn.learn()

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/168 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

epoch:   1 train loss: 0.691772 train f1: 0.041557 valid loss: 0.682525 valid f1: 0.007731


  0%|          | 0/168 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [10]:
model

Sequential(
  (0): Sig2Spec(
    (melspec): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
    (p2db): AmplitudeToDB()
  )
  (1): ResNet(
    (conv1): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2

In [12]:
model[-1]

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): SplAtConv2d(
        (conv): Conv2d

In [22]:
model[1].fc

Linear(in_features=2048, out_features=1000, bias=True)

In [14]:
(300000 / 30) / 64

156.25